In [ ]:
# NOTE: POTENTIALLY ERRONEOUS CODE, UNMAINTAINED

In [ ]:
# DERIVE THE EOMs SYMBOLICALLY -------------------------------------------------------------------------------------

# import libraries
import sympy as sym
import numpy as np
import multiprocessing as mp #multiprocessing, just for the simplification steps
import time # for benchmarking different models

sym.init_printing()
from IPython.display import display #for pretty printing

# define non--program-critical functions
def log_time(t_start=None, msg=""):
    t_end = time.perf_counter()
    if t_start == None: print("Timer initialized")
    else: print(msg + f": {t_end - t_start:.6f} sec")
    return t_end

def mp_trigsimp(m):
    mp_data = [(m[r,c]) for r in range(m.rows) for c in range(m.cols)]
    if __name__ == '__main__':
        p = mp.Pool()
        results = p.map(sym.trigsimp, mp_data)
        p.close()
        p.join()
    return sym.Matrix(results).reshape(m.rows,m.cols)

# ------------------------------------------------------------------------------------------------------------------
# create symbolic variables

# system parameters 
g = sym.symbols('g')
m1, l1, In1, d1 = sym.symbols(['m_{1}','l_{1}','I_{1}','d_{1}'])
m2, l2, In2, d2 = sym.symbols(['m_{2}','l_{2}','I_{2}','d_{2}'])
m3, l3, In3, d3 = sym.symbols(['m_{3}','l_{3}','I_{3}','d_{3}'])
m4, l4, In4, d4 = sym.symbols(['m_{4}','l_{4}','I_{4}','d_{4}'])



# generalized coordinates
x  ,  dx,  ddx = sym.symbols(['x'          ,'\dot{x}'          ,'\ddot{x}'])
y  ,  dy,  ddy = sym.symbols(['y'          ,'\dot{y}'          ,'\ddot{y}'])
th1,dth1,ddth1 = sym.symbols(['\\theta_{1}','\dot{\\theta}_{1}','\ddot{\\theta}_{1}'])
th2,dth2,ddth2 = sym.symbols(['\\theta_{2}','\dot{\\theta}_{2}','\ddot{\\theta}_{2}'])
th3,dth3,ddth3 = sym.symbols(['\\theta_{3}','\dot{\\theta}_{3}','\ddot{\\theta}_{3}'])
th4,dth4,ddth4 = sym.symbols(['\\theta_{4}','\dot{\\theta}_{4}','\ddot{\\theta}_{4}'])

q   = sym.Matrix([  [x],  [y],  [th1],  [th2],  [th3],  [th4]])
dq  = sym.Matrix([ [dx], [dy], [dth1], [dth2], [dth3], [dth4]])
ddq = sym.Matrix([[ddx],[ddy],[ddth1],[ddth2],[ddth3],[ddth4]])


# ------------------------------------------------------------------------------------------------------------------
# functions, # don't ask, they work for the purposes of this program
def WeirdRot(v, th):
    # weird rotation function for working with [x; y; th] vectors
    # usage: gives R_1^0 transformation, and adds the angle of rotation to result
    R = sym.Matrix([[sym.cos(th), -sym.sin(th), 0],
                    [sym.sin(th),  sym.cos(th), 0],
                    [       0,        0, 1]])
    S = sym.Matrix([[0],[0],[th]])
    return R*v + S

def WeirdAdd(v1, v2):
    # returns the sum of two vectors of the form [x; y; th]
    # both in the same ref. frame, preserving only the first angle
    v3 = sym.Matrix([[1],[1],[0]])
    return v1 + sym.matrix_multiply_elementwise(v3, v2)

def WeirdTrans(v1, ang1, off1, ang2=0, off2=sym.Matrix([[0],[0],[0]])):
    # ^ & ^ just like me
    # transforms [x,y,th] vector to inertial frame using position offsets and
    # rotation angles as specified
    rn_m = WeirdAdd(WeirdRot(v1, ang1),off1)
    rn_0 = WeirdAdd(WeirdRot(rn_m, ang2),off2)
    return rn_0


# ------------------------------------------------------------------------------------------------------------------
# mass positions, foot positions
r1_1 = sym.Matrix([[0],[l1*d1],[0]]) #sanity check
r1_0 = WeirdTrans(r1_1, th1, sym.Matrix([[x],[y],[0]]))

r2_2 = sym.Matrix([[0],[-l2*d2],[0]])
r2_0 = WeirdTrans(r2_2, th2+th1, sym.Matrix([[x],[y],[0]])) #sanity check

r3_3 = sym.Matrix([[0],[-l3*d3],[0]])
r3_0 = WeirdTrans(r3_3, th3, r2_2*(1-d2)/d2, th1+th2, r2_0)

r4_4 = sym.Matrix([[l4*d4],[0],[0]]) # could define in x, but for consistency -y is fine
r4_0 = WeirdTrans(r4_4, th4, r3_3*(1-d3)/d3, th1+th2+th3, r3_0)


sym0     = sym.Matrix([[0] ,[0]      ,[0]])
rhip_0   = WeirdTrans(sym0 , th2+th1 , sym.Matrix([[x],[y],[0]])) #sanity check
rknee_0  = WeirdTrans(sym0 , th3     , r2_2*(1-d2)/d2           , th1+th2     , r2_0) #sanity check
rankle_0 = WeirdTrans(sym0 , th4     , r3_3*(1-d3)/d3           , th1+th2+th3 , r3_0)



rfoot_4 = sym.Matrix([[l4],[0],[0]])
rfoot_0 = WeirdTrans(rfoot_4, th4, r3_3*(1-d3)/d3, th1+th2+th3, r3_0) #divide by d3 to get full link length

# hopefully these speed up later simplifications
# update: simplifying here means no simplification needed in dr below (tested for dr3 and dr4).
# update 2: halves the simplification time of T
process_timer = log_time()
r2_0 = sym.trigsimp(r2_0)
r3_0 = sym.trigsimp(r3_0)
r4_0 = sym.trigsimp(r4_0)
rfoot_0 = sym.trigsimp(rfoot_0)
process_timer = log_time(process_timer, "Simplified 'r's")

# mass velocities, foot velocity
dr1 = r1_0.jacobian(q)*dq
dr2 = r2_0.jacobian(q)*dq
dr3 = r3_0.jacobian(q)*dq
dr4 = r4_0.jacobian(q)*dq
drfoot = rfoot_0.jacobian(q)*dq



# ------------------------------------------------------------------------------------------------------------------
# kinetic and potential energy
def Ep(m, r):
    return sym.Matrix([m*g*r[1]])

def Ek(m, In, dr):
    InM = sym.Matrix([[m,0,0],[0,m,0],[0,0,In]])
    return 0.5*dr.T*InM*dr

V = Ep(m1,r1_0) + Ep(m2,r2_0) + Ep(m3,r3_0) + Ep(m4,r4_0)

T = sym.Matrix([Ek(m1,In1,dr1), Ek(m2,In2,dr2), Ek(m3,In3,dr3), Ek(m4,In4,dr4)])
T = mp_trigsimp(T)
T = sum(T[i] for i in range(len(T)))
T = sym.Matrix([T])
process_timer = log_time(process_timer, "Simplified T")


# ------------------------------------------------------------------------------------------------------------------
# forces & torques
tau1, tau2, tau3, GRFx, GRFy = sym.symbols(['\\tau_{1}','\\tau_{2}','\\tau_{3}','G_x','G_y'])

tau1_l1 = sym.Matrix([[0],[0],[-tau1]])
tau1_l2 = sym.Matrix([[0],[0], [tau1]])

tau2_l2 = sym.Matrix([[0],[0],[-tau2]])
tau2_l3 = sym.Matrix([[0],[0], [tau2]])

tau3_l3 = sym.Matrix([[0],[0],[-tau3]])
tau3_l4 = sym.Matrix([[0],[0], [tau3]])

# GRF_r  = WeirdRot((1-d4)*rfoot_4, th1+th2+th3+th4)
# GRFth = GRF_r[0]*GRFy - GRF_r[1]*GRFx
GRF_l4 = sym.Matrix([[GRFx],[GRFy],[0]])


Qtau1 = r1_0.jacobian(q).T*tau1_l1 + r2_0.jacobian(q).T*tau1_l2
Qtau2 = r2_0.jacobian(q).T*tau2_l2 + r3_0.jacobian(q).T*tau2_l3
Qtau3 = r3_0.jacobian(q).T*tau3_l3 + r4_0.jacobian(q).T*tau3_l4
QGRF  = rfoot_0.jacobian(q).T*GRF_l4

Qtot = Qtau1 + Qtau2 + Qtau3 + QGRF

# ------------------------------------------------------------------------------------------------------------------
# Mass, Gravity, Coriolis matrices, EOM
LgM = sym.hessian(T, dq)
LgM = mp_trigsimp(LgM)
process_timer = log_time(process_timer, "Simplified LgM")

LgG = V.jacobian(q)

dM = LgM.reshape(LgM.rows * LgM.cols, 1)
dM = dM.jacobian(q)*dq
dM = dM.reshape(LgM.rows, LgM.cols)
Tj = T.jacobian(q)
LgC = dM*dq - Tj.T
LgC = mp_trigsimp(LgC)

# tested (dM then LgC) ,(Tj then LgC), (dM, Tj, then LgC), (LgC)
# simplify (LgC) is fastest

process_timer = log_time(process_timer, "Simplified LgC")

EOM = LgM*ddq + LgC + LgG.T - Qtot
EOM = mp_trigsimp(EOM)
process_timer = log_time(process_timer, "Simplified EOM")
#EOM0 = LgM*ddq - QGRF

In [ ]:
# # LAMBDIFY ------------------------------------------------------------------------------------------------------------------    
from pyomo.environ import*
from pyomo.opt import SolverFactory
from pyomo.opt import SolverStatus, TerminationCondition

func_map = {'sin':sin, 'cos':cos}


sym_list = [g,
            m1 , m2, m3, m4,
            l1 , l2, l3, l4,
            In1,In2,In3,In4,
            d1 , d2, d3, d4,
            x  ,  y,  th1,  th2,  th3,  th4,
            dx , dy, dth1, dth2, dth3, dth4,
            ddx,ddy,ddth1,ddth2,ddth3,ddth4,
            tau1,tau2,tau3,
            GRFx,GRFy]

lamb_EOM  = {}
#lamb_EOM0 = {}

# DOFs
DOFs  = ['x','y','th1','th2','th3','th4']
TDOFs = ['x','y']


links = ['b','l1','l2','l3']
joints = ['hip','knee','ankle']

for dof_i, dof in enumerate(DOFs):
    lamb_EOM[dof]  = sym.lambdify(sym_list, EOM[dof_i],modules = [func_map])
    #lamb_EOM0[dof] = sym.lambdify(sym_list,EOM0[dof_i],modules = [func_map])
    
lamb_footp = {}
lamb_footv = {}

lamb_massp = {}
lamb_jointp = {}

for dof_i, dof in enumerate(TDOFs):
    lamb_footp[dof]  = sym.lambdify(sym_list,rfoot_0[dof_i],modules = [func_map])
    lamb_footv[dof]  = sym.lambdify(sym_list, drfoot[dof_i],modules = [func_map])
    
    lamb_massp.update({('b',dof)  : sym.lambdify(sym_list,r1_0[dof_i],modules = [func_map])})
    lamb_massp.update({('l1',dof) : sym.lambdify(sym_list,r2_0[dof_i],modules = [func_map])})
    lamb_massp.update({('l2',dof) : sym.lambdify(sym_list,r3_0[dof_i],modules = [func_map])})
    lamb_massp.update({('l3',dof) : sym.lambdify(sym_list,r4_0[dof_i],modules = [func_map])})

    lamb_jointp.update({('hip',dof)   : sym.lambdify(sym_list,  rhip_0[dof_i],modules = [func_map])})
    lamb_jointp.update({('knee',dof)  : sym.lambdify(sym_list, rknee_0[dof_i],modules = [func_map])})
    lamb_jointp.update({('ankle',dof) : sym.lambdify(sym_list,rankle_0[dof_i],modules = [func_map])})

In [ ]:
# PARAMETERS:
mass = {'b': 0.5,
       'l1': 0.2,
       'l2': 0.2,
       'l3': 0.1}

length = {'b': 1,
         'l1': 0.5,
         'l2': 0.5,
         'l3': 0.3}

dCOM = {'b': 0.5,
       'l1': 0.4,
       'l2': 0.4,
       'l3': 0.2}

inertia = {'b': mass['b']*length['b']**2/12,
          'l1': mass['l1']*length['l1']**2/12,
          'l2': mass['l2']*length['l2']**2/12,
          'l3': mass['l3']*length['l3']**2/12}

def get_var_list(m,n):
    var_list = [m.g]+\
               [m.m[l] for l in links]+\
               [m.len[l] for l in links]+\
               [m.In[l] for l in links]+\
               [m.dCOM[l] for l in links]+\
               [m.q[n,dof] for dof in DOFs]+\
               [m.dq[n,dof] for dof in DOFs]+\
               [m.ddq[n,dof] for dof in DOFs]+\
               [m.tau[n,j] for j in joints]+\
               [m.GRF[n,dof] for dof in TDOFs]

    return var_list

In [ ]:
def create_model(N,hm,mu):
    m = ConcreteModel()
    
    #SETS---------------------------------
    m.N =RangeSet(N)
    m.L =Set(initialize = links)
    m.J = Set(initialize = joints)
    m.DOF = Set(initialize= DOFs)
    m.TDOF = Set(initialize = TDOFs)
    
    #PARAMS-------------------------------
    m.g = Param(initialize = 9.81)
    m.mu = Param(initialize = mu)
    
    m.m = Param(m.L, initialize = mass)
    m.len = Param(m.L, initialize = length)
    m.In = Param(m.L, initialize = inertia)
    m.dCOM = Param(m.L, initialize = dCOM)
    
    signs = ['ps','ng']
    m.sgn = Set(initialize = signs)
    m.ground_constraints = ['contact','sliding_ps','sliding_ng']
    
    #VARIABLES----------------------------
    #gen coordinates
    m.q = Var(m.N,m.DOF)
    m.dq = Var(m.N,m.DOF)
    m.ddq = Var(m.N,m.DOF)
    
    #forces
    m.tau = Var(m.N,m.J)
    m.GRF = Var(m.N,m.TDOF)

# ------------------------------------------------------------------------------------------------------------------
    #TIME AND INTEGRATION

    # timestep
    m.hm = Param(initialize = hm, mutable = True) # master timestep
    m.h = Var(m.N, bounds = (0.8,1.0))
    
    # Integration constraints 
    def BwEuler_p(m,n,dof): # for positions
        if n > 1:
            return m.q[n,dof] == m.q[n-1,dof] + hm*m.h[n]*m.dq[n-1,dof]
        else:
            return Constraint.Skip 
    m.integrate_p = Constraint(m.N, m.DOF, rule = BwEuler_p)

    def BwEuler_v(m,n,dof): # for velocities
        if n > 1:
            return m.dq[n,dof] == m.dq[n-1,dof] + hm*m.h[n]*m.ddq[n-1,dof]
        else:
            return Constraint.Skip 
    m.integrate_v = Constraint(m.N, m.DOF, rule = BwEuler_v)
    
    def dynamics(m,n,dof):
        var_list = get_var_list(m,n)
        temp = lamb_EOM[dof](*var_list) == 0
#         if temp in [None, True, False]:
#             return Constraint.Skip
#         else:
        return temp
    m.dynamics = Constraint(m.N,m.DOF, rule = dynamics)


# ------------------------------------------------------------------------------------------------------------------
    # Link Bounds ------------------------
    for n in range(1,N+1):
        m.GRF[n,'y'].setlb(0)
        m.tau[n,'hip'].setlb(-20)
        m.tau[n,'hip'].setub(20)
        m.tau[n,'knee'].setlb(-10)
        m.tau[n,'knee'].setub(10)
        m.tau[n,'ankle'].setlb(-10)
        m.tau[n,'ankle'].setub(10)

        m.q[n,'x'].setlb(0.0)
        m.q[n,'y'].setlb(0.0)
        m.q[n,'th1'].setlb(-np.pi/2)
        m.q[n,'th1'].setub(np.pi/2)
        # https://www.researchgate.net/figure/The-range-of-movements-in-the-hip-joint-a-flexion-extension-10-0-130_fig4_326636749
        m.q[n,'th2'].setlb(-np.pi/2/9)
        m.q[n,'th2'].setub(np.pi*3/4)
        #https://x10therapy.com/knee-range-of-motion/
        m.q[n,'th3'].setlb(-np.pi*3/4)
        m.q[n,'th3'].setub(0)
        #https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4994968/
        m.q[n,'th4'].setlb(-np.pi/3) # INSANITY check
        m.q[n,'th4'].setub(np.pi/12)

    #Ground Constraints ------------------
    m.footp = Var(m.N, m.TDOF)
    def def_footp(m,n,dof):
        var_list = get_var_list(m,n)
        if dof == 'y':
            m.footp[n,dof].setlb(0)
        return m.footp[n,dof] == lamb_footp[dof](*var_list)
    m.def_footp = Constraint(m.N, m.TDOF, rule = def_footp)
        
    m.footv = Var(m.N, m.TDOF, m.sgn, bounds = (0.0,None))
    def def_footv(m,n,dof,sgn):
        var_list = get_var_list(m,n)
        return m.footv[n,dof,'ps'] - m.footv[n,dof,'ng'] == lamb_footv[dof](*var_list)
    m.def_footv = Constraint(m.N, m.TDOF, m.sgn, rule = def_footv)
    
    # Joint and Mass Positions -----------
    m.jointp = Var(m.N, m.J, m.TDOF, bounds = (0.0,None))
    def def_jointp(m,n,j,dof):
        var_list = get_var_list(m,n)
        return m.jointp[n,j,dof] == lamb_jointp[(j,dof)](*var_list)
    m.def_jointp = Constraint(m.N, m.J, m.TDOF, rule = def_jointp)
 
    m.massp = Var(m.N, m.L, m.TDOF, bounds = (0.0,None))
    def def_massp(m,n,l,dof):
        var_list = get_var_list(m,n)
        return m.massp[n,l,dof] == lamb_massp[(l,dof)](*var_list)
    m.def_massp = Constraint(m.N, m.L, m.TDOF, rule = def_massp)





    m.alpha = Var(m.N, bounds = (0,1))
    def friction(m,n):
        if n == 1:
            return Constraint.Skip
        return m.GRF[n,'x'] == (1 - 2*m.alpha[n])*m.mu*m.GRF[n,'y']
    m.friction = Constraint(m.N, rule = friction)
    
    m.ground_penalty = Var(m.N, m.ground_constraints, bounds = (0.0,None))
    def ground_complementarity(m,n,gc):
        if n == 1:
            return Constraint.Skip
        if gc == 'contact':
            A = m.GRF[n,'y']
            B = m.footp[n,'y'] + m.footv[n,'x','ps'] + m.footv[n,'x','ng']
            
        if gc == 'sliding_ps':
            A = 1 - m.alpha[n]
            B = m.footv[n,'x','ps']
            
        if gc == 'sliding_ng':
            A = m.alpha[n]
            B = m.footv[n,'x','ng']
        
        return A*B <= m.ground_penalty[n,gc]
    m.ground_complementarity = Constraint(m.N, m.ground_constraints, rule = ground_complementarity)
 




    # OBJECTIVES -----------------------------------------------------------------------------------------------------------
    def MinPenalty(m):
        return sum([m.ground_penalty[n,gc] for n in range(1,N+1) for gc in m.ground_constraints])
    m.MinPenalty = Objective(rule = MinPenalty)

#     def MinPenAndTorque(m):
#         penalty_sum = sum([m.ground_penalty[n,gc] for n in range(1,N+1) for gc in m.ground_constraints])
#         torque_sum  = sum([m.tau[n,j]**2          for n in range(1,N+1) for j in m.J])
#         scale_factor = 1e6
#         return scale_factor*penalty_sum + torque_sum
#     m.MinPenAndTorque = Objective(rule = MinPenAndTorque)
    
    return m


In [ ]:
def init_opt(max_iter, max_time, solver): # initializes solver
    opt = SolverFactory('ipopt') # standard issue, garden variety ipopt
    opt.options["linear_solver"] = solver
    opt.options["print_level"] = 5 # prints a log with each iteration (you want to this - it's the only way to see progress.)
    opt.options["max_iter"] = max_iter # maximum number of iterations
    opt.options["max_cpu_time"] = max_time # maximum cpu time in seconds
    opt.options["Tol"] = 1e-6 # the tolerance for feasibility. Considers constraints satisfied when they're within this margin.
    
    opt.options["OF_acceptable_obj_change_tol"] = 1e-4
    opt.options["OF_ma86_scaling"] = 'none'
    
    return opt


In [ ]:
# HIGH DROP -----------------------------------------------------------------------------------------------------------------

def high_drop(m):
    # initial condition
    for dof in DOFs:
        #m.ddq[1,dof].fix(0)
        m.dq[ 1,dof].fix(0) # rest
        if dof not in ['y','th4']:
            m.q[1,dof].fix(0) # neutral posture
        m.q[1,'th4'].fix(np.pi/3)
    m.footy[1].fix(20)
    
    # during
    for n in range(1,N+1):
        for j in joints:
            m.tau[n,j].fix(0)

    return m

In [ ]:
# LOW DROP ------------------------------------------------------------------------------------------------------------------

def low_drop(m):
    # initial condition
    for dof in DOFs:
        m.ddq[1,dof].fix(0)
        m.dq[ 1,dof].fix(0) # rest
        if dof not in ['y','th4']:
            m.q[1,dof].fix(0) # neutral posture
        m.q[1,'th4'].fix(np.pi/2)
    m.footy[1].setlb(0.1) # start near the ground

    # during
    for n in range(1,N+1):
        for j in joints:
            m.tau[n,j].fix(0) # no forces
        if n > 15:
            m.footy[15].fix(0) # force it to land
    return m

In [ ]:
# # # STANDING ------------------------------------------------------------------------------------------------------------------
def standing(m):
    # initial condition
    for dof in DOFs:
        m.dq[1,dof].fix(0) # rest
        m.q[ 1,dof].fix(0) # neutral posture
    m.q[1,  'y'].unfix()
    m.q[1,'th4'].unfix()

    # during
    for n in range(1,N+1):
        m.footy[n].fix(0) # must stay grounded for the full time

    # final condition (hold)
    for n in range(N-20,N+1):
        for dof in DOFs:
            m.dq[ n,dof].fix(0)
        m.q[n,'th1'].fix(0)
    
    return m

In [ ]:
# BOUNCE --------------------------------------------------------------------------------------------------------------------

def bounce(m):
    for dof in DOFs:
        m.ddq[1,dof].fix(0)
        m.dq[ 1,dof].fix(0)
    m.q[1,'x'].fix(0)
    #    m.q[ 1,dof].fix(0)
    #m.q[1,  'y'].unfix()
    #m.q[1,'th4'].unfix()
    
    m.footy[1].fix(0)
        
    # middle
    m.footy[15].setlb(0.2)
    for n in range(1,N+1):
        m.q[n,'th1'].setlb(-np.pi/10)
        m.q[n,'th1'].setub(np.pi/10)
    
    # end
    
    for n in range(N-5,N+1):
        for dof in DOFs:
            m.dq[n,dof].fix(0)
        m.footy[N].fix(0)
        m.q[n,'th1'].fix(0)

    return m

In [ ]:
# HIGH DROP + NUDGE ---------------------------------------------------------------------------------------------------------

def shoot(m):
    # initial condition
    for dof in DOFs:
        m.ddq[1,dof].fix(0)
        m.dq[ 1,dof].fix(0) # rest
        m.q[  1,dof].fix(0) # neutral posture
    m.dq[1,'x','b'].fix(3)
    m.q[ 1,'y','b'].unfix()
    for s in sides:
        m.footp[1,'y',s].setlb(5)
    
    # final
    for n in range(N-3,N+1):
        for dof in DOFs:
            #m.ddq[n,dof].fix(0)
            m.dq[ n,dof].fix(0)
        for s in sides:
            m.footp[n,'y',s].fix(0)

    return m

In [ ]:
# JUMP ---------------------------------------------------------------------------------------------------------------------
# works with new (gc + min) tau objective function (2024/20/16)
def jump(m):
    for dof in DOFs:
        m.ddq[1,dof].fix(0)
        m.dq[ 1,dof].fix(0)
    m.q[1,'x'].fix(0)
    m.footp[1,'y'].fix(0)
        
    # middle
    m.footp[int(N/2),'y'].setlb(0.6) # jump height
    for n in range(1,N+1):
        m.footp[n,'x'].setlb(-0.5) # don't throw feet up away from body
        m.footp[n,'x'].setub(0.5)
        m.q[n,'th1'].setub(0) # don't lean backwards # doesn't matter that it's in the sides loop
#             m.q[n,'th4'].setlb(-np.pi/3) # don't stand with feet facing directly downwards
    
    # end
    for n in range(N-5,N+1):
        for dof in DOFs:
            m.dq[n,dof].fix(0)
        m.q[n,'x'].fix(0)
        m.footp[n,'y'].fix(0)
                
    return m

In [ ]:
from IPython.display import display, HTML #for pretty printing
display(HTML("<style>.jp-CodeCell.jp-mod-outputsScrolled .jp-Cell-outputArea { max-height: 32em; }</style>"))

N = 100
hm = 0.05
mu = 1.0
m = create_model(N,hm,mu)

# m = high_drop(m)
# m = low_drop(m)
# m = stand(m)
m = jump(m)
# m = shoot(m)
# m = twostep(m)
# m = onestep(m)
# m = run(m)

opt = init_opt(30000,3000,'ma97')
process_timer = log_time()
results = opt.solve(m,tee=True)
process_timer = log_time(process_timer, "Solved")

In [ ]:
# if the problem is infeasible, this is how you can see which constraints weren't satisfied
from pyomo.util.infeasible import log_infeasible_constraints
log_infeasible_constraints(m)

In [ ]:
#animate it
import matplotlib.pyplot as plt
import matplotlib.animation as ani
from IPython.display import HTML
%matplotlib inline

fig1, ax1 = plt.subplots(1,1) #create axes
ax1.set_aspect('equal')

xmax = np.max([m.q[n,'x'].value for n in range(1,N+1)])
ymax = np.max([m.q[n,'y'].value for n in range(1,N+1)]) + m.len['b']

def plot_model(i,m,ax): #update function for animation
    ax.clear()   
    ax.set_xlim([-2 + m.q[1,'x'].value, 2 + m.q[1,'x'].value]) # adjust limits to solution
    ax.set_ylim([0,ymax+0.5])
    
    #plot body
    draw_width = 0.05
    body_xtl = m.q[i,'x'].value - (m.len[('b')])*sin(m.q[i,'th1'].value) - draw_width*cos(m.q[i,'th1'].value)
    body_ytl = m.q[i,'y'].value + (m.len[('b')])*cos(m.q[i,'th1'].value) - draw_width*sin(m.q[i,'th1'].value)
    body_xtr = m.q[i,'x'].value - (m.len[('b')])*sin(m.q[i,'th1'].value) + draw_width*cos(m.q[i,'th1'].value)
    body_ytr = m.q[i,'y'].value + (m.len[('b')])*cos(m.q[i,'th1'].value) + draw_width*sin(m.q[i,'th1'].value)
    body_xbl = m.q[i,'x'].value - draw_width*cos(m.q[i,'th1'].value)
    body_ybl = m.q[i,'y'].value - draw_width*sin(m.q[i,'th1'].value)
    body_xbr = m.q[i,'x'].value + draw_width*cos(m.q[i,'th1'].value)
    body_ybr = m.q[i,'y'].value + draw_width*sin(m.q[i,'th1'].value)
    ax.plot([body_xbl,body_xtl,body_xtr,body_xbr,body_xbl],[body_ybl,body_ytl,body_ytr,body_ybr,body_ybl],color='xkcd:blue')
    
    #plot thigh
    leg1_xt = m.jointp[i, 'hip','x'].value
    leg1_yt = m.jointp[i, 'hip','y'].value
    leg1_xb = m.jointp[i,'knee','x'].value
    leg1_yb = m.jointp[i,'knee','y'].value
    ax.plot([leg1_xt,leg1_xb],[leg1_yt,leg1_yb],color='xkcd:green')

    #plot shank
    leg2_xt = m.jointp[i, 'knee','x'].value
    leg2_yt = m.jointp[i, 'knee','y'].value
    leg2_xb = m.jointp[i,'ankle','x'].value
    leg2_yb = m.jointp[i,'ankle','y'].value
    ax.plot([leg2_xt,leg2_xb],[leg2_yt,leg2_yb],color='xkcd:black')

    #plot foot
    leg3_xt = m.jointp[i,'ankle','x'].value
    leg3_yt = m.jointp[i,'ankle','y'].value
    leg3_xb =  m.footp[i,        'x'].value
    leg3_yb =  m.footp[i,        'y'].value
    ax.plot([leg3_xt,leg3_xb],[leg3_yt,leg3_yb],color='xkcd:red')
        
    
        
    # plot COMs
    COMx = 0
    COMy = 0
    Mtot = 0
    for l in links:
        COMx = COMx + m.massp[i,l,'x'].value*m.m[l]
        COMy = COMy + m.massp[i,l,'y'].value*m.m[l]
        Mtot = Mtot + m.m[l]
        #ax.plot([m.massp[i,l,'x'].value],[m.massp[i,l,'y'].value], color='xkcd:blue',marker='o')
    COMx = COMx / Mtot
    COMy = COMy / Mtot
    ax.plot([COMx],[COMy], color='xkcd:red',marker='o')
update = lambda i: plot_model(i,m,ax1) #lambdify update function

animate = ani.FuncAnimation(fig1,update,range(1,N+1),interval = 50,repeat=True)

HTML(animate.to_html5_video()) #you need to convert the animation to HTML5 to embed it in the notebook

In [ ]:
m.footy.pprint()